## KICKOFF Project
### Kickoff is a project that predict the results of matches in the FIFA World Cup 2022 

## Section1: Business Understanding
    We will ansewr three question on the end of this notebook
    
   ### Question 1: Which teams have the better odds to domenate their groups according to KICKOFF?
   ### Question 2: Which team will be the black horse in the World Cup ?
   ### Question 3: Who will win the World Cup according to KICKOFF?


## Section2: Data Understanding

### Step.No1: Gathering Data 
##### We will import the datasets that we will going to use
###### match_df : it's has all the international matches from 1993 until 6/2022.
###### rank_df : it's has the FIFA rank for all nations. (FIFA rank: is an offical rank form FIFA based on the proformance of the teams).
###### worldcup_match_df : it's has all the matches that will happen in Qatar world cup 2022.

In [28]:
# import pandas
import pandas as pd
match_df=pd.read_csv('international_matches.csv')
rank_df=pd.read_csv('fifa_ranking_2022-10-06.csv')
worldcup_match_df = pd.read_csv('matchs-schudule.csv',sep=';')

In [29]:
rank_df.head()

,team,team_code,association,rank,previous_rank,points,previous_points
0,Brazil,BRA,CONMEBOL,1,1,1841.30,1837.56
1,Belgium,BEL,UEFA,2,2,1816.71,1821.92
2,Argentina,ARG,CONMEBOL,3,3,1773.88,1770.65
3,France,FRA,UEFA,4,4,1759.78,1764.85
4,England,ENG,UEFA,5,5,1728.47,1737.46


In [30]:
worldcup_match_df.head()

,match,date,country1,coutry2,phase
0,1,21/11/2022,Qatar,Ecuador,group matches
1,2,21/11/2022,Senegal,Netherlands,group matches
2,3,21/11/2022,England,Iran,group matches
3,4,21/11/2022,USA,Wales,group matches
4,5,22/11/2022,France,Australia,group matches


### we will create new dataframe for the teams that will be in Qatar 2022

In [31]:
teams_data=[['Qatar', 'A'], ['Ecuador', 'A'], ['Senegal', 'A'],['Netherlands','A'],
            ['England','B'],['Iran','B'],['USA','B'],['Wales','B'],
            ['Argentina','C'],['Saudi Arabia','C'],['Mexico','C'],['Poland','C'],
            ['France','D'],['Denmark','D'],['Tunisia','D'],['Australia','D'],
            ['Spain','E'],['Costa Rica','E'],['Germany','E'],['Japan','E'],
            ['Belgium','F'],['Canada','F'],['Morocco','F'],['Croatia','F'],
            ['Brazil','G'],['Serbia','G'],['Switzerland','G'],['Cameroon','G'],
            ['Portugal','H'],['Ghana','H'],['Uruguay','H'],['South Korea','H']]

teams_df = pd.DataFrame(teams_data, columns=['Team', 'Group'])
teams_df.head()

,Team,Group
0,Qatar,A
1,Ecuador,A
2,Senegal,A
3,Netherlands,A
4,England,B


## Section3: Prepare Data

### : Cleaning Data 
##### We will need to clean the data because there is some column we don't need.
##### Before deleting any columns there are some columns we need we will add it.

In [32]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
date                             23921 non-null object
home_team                        23921 non-null object
away_team                        23921 non-null object
home_team_continent              23921 non-null object
away_team_continent              23921 non-null object
home_team_fifa_rank              23921 non-null int64
away_team_fifa_rank              23921 non-null int64
home_team_total_fifa_points      23921 non-null int64
away_team_total_fifa_points      23921 non-null int64
home_team_score                  23921 non-null int64
away_team_score                  23921 non-null int64
tournament                       23921 non-null object
city                             23921 non-null object
country                          23921 non-null object
neutral_location                 23921 non-null bool
shoot_out                        23921 non-null object
home_team_result   

### For match_df we will add these following columns that we think it will help us:

##### rank_difference: the rank diffrence between the two teams that play against each other.
##### average_rank: the average rank of the two teams.
##### point_difference: the rank point difference between the two teams.
##### is_offical: is the match between these two team friendly or not, it will be Bool as if it friendly it will be False otherwise will be True.
##### is_worldcup:is the match between these two team in the world cup or not, it will be Bool as if it is in the world cup it will be True otherwise will be False.
##### score_difference: we wiil use it to know the winner is it the home team or not.(we will not going to use it in the model)
##### is_won: based on score_difference if it is >0 it will be True otherwise it will be false.


In [33]:
match_df['rank_difference'] = match_df['home_team_fifa_rank'] - match_df['away_team_fifa_rank']
match_df['average_rank'] = (match_df['home_team_fifa_rank'] + match_df['away_team_fifa_rank'])/2
match_df['point_difference'] = match_df['home_team_total_fifa_points'] - match_df['away_team_total_fifa_points']
match_df['is_stake'] = match_df['tournament'] != 'Friendly'
match_df['is_worldcup'] = 'FIFA World Cup' in match_df['tournament']
match_df['score_difference'] = match_df['home_team_score'] - match_df['away_team_score'] 
match_df['is_won'] = match_df['score_difference'] > 0 

In [34]:
match_df.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,rank_difference,average_rank,point_difference,is_stake,is_worldcup,score_difference,is_won
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,NaN,NaN,NaN,37,40.5,0,True,False,2,True
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,NaN,NaN,NaN,-6,11.0,0,False,False,0,False
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,NaN,NaN,NaN,-59,64.5,0,True,False,5,True
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,NaN,NaN,NaN,-21,75.5,0,False,False,1,True
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,NaN,NaN,NaN,62,36.0,0,True,False,-2,False


In [35]:
#change some teams name
match_df = match_df.replace({"IR Iran": "Iran", "Korea Republic" : "South Korea"})
rank_df = rank_df.replace({"IR Iran": "Iran", "Korea Republic" : "South Korea"})

In [36]:
# We will remove the teams that will not play in the World Cup from rank_df
rank_df = rank_df.loc[(rank_df['team'].isin(teams_df['Team'].unique()))]
rank_df=rank_df.set_index(['team'])
rank_df.index.unique()

Index(['Brazil', 'Belgium', 'Argentina', 'France', 'England', 'Spain',
       'Netherlands', 'Portugal', 'Denmark', 'Germany', 'Croatia', 'Mexico',
       'Uruguay', 'Switzerland', 'USA', 'Senegal', 'Wales', 'Iran', 'Serbia',
       'Morocco', 'Japan', 'Poland', 'South Korea', 'Tunisia', 'Costa Rica',
       'Australia', 'Canada', 'Cameroon', 'Ecuador', 'Qatar', 'Saudi Arabia',
       'Ghana'],
      dtype='object', name='team')

### Now we will drop the columns that we no longer need.
#### we droped these columns because they have so many missing value and they do not add much to our results and some of the coulmns we used it to find something like we used home_team_fifa_rank and away_team_fifa_rank to find the average_rank and now we do not need them any more. Also their are columns that do not have any benfit to us like shoot_out and home_team_goalkeeper_score etc.

In [37]:
match_df.drop(['date', 'home_team_continent', 'away_team_continent', 'home_team_fifa_rank', 'away_team_fifa_rank',
               'home_team_total_fifa_points','away_team_total_fifa_points','home_team_score','away_team_score',
               'tournament','city','country','neutral_location','shoot_out','home_team_result','home_team_goalkeeper_score',
               'away_team_goalkeeper_score','home_team_mean_defense_score','home_team_mean_offense_score','home_team_mean_midfield_score',
               'away_team_mean_defense_score','away_team_mean_offense_score','away_team_mean_midfield_score','score_difference'], axis=1,inplace=True)

In [38]:
match_df.head()

,home_team,away_team,rank_difference,average_rank,point_difference,is_stake,is_worldcup,is_won
0,Bolivia,Uruguay,37,40.5,0,True,False,True
1,Brazil,Mexico,-6,11.0,0,False,False,False
2,Ecuador,Venezuela,-59,64.5,0,True,False,True
3,Guinea,Sierra Leone,-21,75.5,0,False,False,True
4,Paraguay,Argentina,62,36.0,0,True,False,False


In [39]:
import matplotlib.pyplot as plt

 ## Section 4: Data Modeling
 
###  Building the model 
##### We will build the model that will predict the results.
###### we will use rank_difference and average_rank and point_difference and is_stake and is_worldcup to train the model.

In [40]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures



from sklearn.ensemble import RandomForestClassifier

In [41]:
X = match_df.loc[:,['average_rank', 'rank_difference', 'point_difference', 'is_stake', 'is_worldcup']]
y = match_df['is_won']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.29, random_state=42)

### We chose random forest as it is the closet alghrothem to what we need.

In [42]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_test, y_test) * 100, 2)
acc_random_forest

64.180000000000007

In [43]:
model = random_forest

In [44]:
import numpy as np

## Simulate the world cup results by using functions
 

In [45]:
#opponents = ['First match \nagainst', 'Second match\n against', 'Third match\n against']
#Building new dataframe named world_cup contains all world cup teams
world_cup = teams_df.set_index(['Team'])
# Adding points column to world_cup dataframe, which we will collect the points for each team
world_cup['points'] = 0
# For this column we will sum the total probabilty for each team
world_cup['total_prob'] = 0
# Here the expected point for each team
world_cup['expected points'] = 0

#world_cup['rank'] = None

country_win_prob = {}

for country in teams_df['Team'].unique():
    country_win_prob[country] = list()

### Here we will build a function to simulate the group stage
##### We will give it the teams particpate in the group stage and it will give us the precentage win for each match.

In [46]:
from itertools import combinations
import numpy as np

# Margin for draw cases:
Margin=0.05

#define a function to use with each stage like Group, Round of 16, etc.
def predictG(stage):
    """
    returns the result of each match in the group stage
    
    input:
        stage: Dataframe of the teams and their groups
        
    """
    for group in stage['Group'].drop_duplicates():
        print('_Starting Group {}:_'.format(group))

        for home, away in combinations(stage.query('Group == "{}"'.format(group)).values, 2):
            print("{} vs. {}: ".format(home[0], away[0]), end='')

            home = home[0]
            away = away[0]

            # Create a row for each match
            row = pd.DataFrame(np.array([[np.nan, np.nan, np.nan, True, True]]), columns=X_train.columns)

            home_rank = rank_df.loc[home, 'rank']
            home_points = rank_df.loc[home, 'points']
            opp_rank = rank_df.loc[away, 'rank']
            opp_points = rank_df.loc[away, 'points']

            row['average_rank'] = (home_rank + opp_rank) / 2
            row['rank_difference'] = home_rank - opp_rank
            row['point_difference'] = home_points - opp_points

            # Model Output
            home_win_prob = model.predict_proba(row)[:,1][0]



            # Saving Model Output
            world_cup.loc[home, 'total_prob'] += home_win_prob
            world_cup.loc[away, 'total_prob'] += 1-home_win_prob

            # Determining Win / Draw / Lose
            points = 0
            if home_win_prob <= 0.5 - Margin :
                print("{} wins with {:.2f}".format(away, 1-home_win_prob))
                world_cup.loc[away, 'points'] += 3
                world_cup.loc[away, 'expected points'] += (1-home_win_prob) * 3
            if home_win_prob > 0.5 - Margin:
                points = 1
            if home_win_prob >= 0.5 + Margin:
                points = 3
                world_cup.loc[home, 'points'] += 3
                world_cup.loc[home, 'expected points'] += home_win_prob * 3
                print("{} wins with {:.2f}".format(home, home_win_prob))
            if points == 1:
                print("Draw")
                world_cup.loc[home, 'points'] += 1
                world_cup.loc[away, 'points'] += 1
                world_cup.loc[home, 'expected points'] += home_win_prob * 1
                world_cup.loc[away, 'expected points'] += (1-home_win_prob) * 1

## Section 5: Evaluate the Results

In [47]:
predictG(teams_df)

_Starting Group A:_
Qatar vs. Ecuador: Qatar wins with 0.58
Qatar vs. Senegal: Senegal wins with 0.80
Qatar vs. Netherlands: Netherlands wins with 0.72
Ecuador vs. Senegal: Senegal wins with 0.74
Ecuador vs. Netherlands: Draw
Senegal vs. Netherlands: Netherlands wins with 0.79
_Starting Group B:_
England vs. Iran: England wins with 0.89
England vs. USA: Draw
England vs. Wales: England wins with 0.85
Iran vs. USA: USA wins with 0.55
Iran vs. Wales: Draw
USA vs. Wales: USA wins with 0.56
_Starting Group C:_
Argentina vs. Saudi Arabia: Argentina wins with 0.74
Argentina vs. Mexico: Draw
Argentina vs. Poland: Argentina wins with 0.65
Saudi Arabia vs. Mexico: Mexico wins with 0.84
Saudi Arabia vs. Poland: Poland wins with 0.82
Mexico vs. Poland: Poland wins with 0.66
_Starting Group D:_
France vs. Denmark: France wins with 0.83
France vs. Tunisia: France wins with 0.93
France vs. Australia: France wins with 0.87
Denmark vs. Tunisia: Denmark wins with 0.68
Denmark vs. Australia: Denmark wins

### Question 1: Which teams have the better odds to domenate their groups according to KICKOFF?
#### As we can see here Belgium and France have the odds to domenate their groups.


In [48]:
for group in teams_df['Group'].drop_duplicates():
    print('___Starting group {}:___'.format(group))
    teams = teams_df.query('Group == "{}"'.format(group)).values
    for team_info in teams:
        team_name = team_info[0]
        print(team_name, ": ", round(world_cup.loc[team_name, "expected points"],2))
    
   

___Starting group A:___
Qatar :  1.74
Ecuador :  0.46
Senegal :  4.62
Netherlands :  5.07
___Starting group B:___
England :  5.74
Iran :  0.5
USA :  3.81
Wales :  0.5
___Starting group C:___
Argentina :  4.71
Saudi Arabia :  0.0
Mexico :  2.98
Poland :  4.43
___Starting group D:___
France :  7.89
Denmark :  4.53
Tunisia :  0.47
Australia :  0.53
___Starting group E:___
Spain :  5.92
Costa Rica :  0.0
Germany :  2.43
Japan :  3.33
___Starting group F:___
Belgium :  7.26
Canada :  0.0
Morocco :  2.55
Croatia :  4.29
___Starting group G:___
Brazil :  6.91
Serbia :  0.98
Switzerland :  2.91
Cameroon :  0.51
___Starting group H:___
Portugal :  6.26
Ghana :  0.0
Uruguay :  2.35
South Korea :  2.31


##### We will create a list for each group so we save in them the top2 teams

In [49]:
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]

world_cup["Rank"] = world_cup["expected points"].rank()
world_cup
  
world_cup.sort_values(["Group","expected points"], inplace = True,ascending=False)
grouped = world_cup.groupby(['Group'])
A = grouped.get_group("A")
A = A.reset_index()
B = grouped.get_group("B")
B = B.reset_index()
C = grouped.get_group("C")
C = C.reset_index()
D = grouped.get_group("D")
D = D.reset_index()
E = grouped.get_group("E")
E = E.reset_index()
F = grouped.get_group("F")
F = F.reset_index()
G = grouped.get_group("G")
G = G.reset_index()
H = grouped.get_group("H")
H = H.reset_index()


##### based on the position of each team it will determine the matches of 16th stage

In [50]:
round16=[[A.iloc[0,0],'R1'],[B.iloc[1,0],'R1'],[B.iloc[0,0],'R2'],[A.iloc[1,0],'R2'],
         [C.iloc[0,0],'R3'],[D.iloc[1,0],'R3'],[D.iloc[0,0],'R4'],[C.iloc[1,0],'R4'],
         [E.iloc[0,0],'R5'],[F.iloc[1,0],'R5'],[F.iloc[0,0],'R6'],[E.iloc[1,0],'R6'],
         [G.iloc[0,0],'R7'],[H.iloc[1,0],'R7'],[H.iloc[0,0],'R8'],[G.iloc[1,0],'R8']]
round_16_df = pd.DataFrame(round16, columns=['Team', 'Group'])
round16

[['Netherlands', 'R1'],
 ['USA', 'R1'],
 ['England', 'R2'],
 ['Senegal', 'R2'],
 ['Argentina', 'R3'],
 ['Denmark', 'R3'],
 ['France', 'R4'],
 ['Poland', 'R4'],
 ['Spain', 'R5'],
 ['Croatia', 'R5'],
 ['Belgium', 'R6'],
 ['Japan', 'R6'],
 ['Brazil', 'R7'],
 ['Uruguay', 'R7'],
 ['Portugal', 'R8'],
 ['Switzerland', 'R8']]

### Now we will create a function named predict
###### it's similar to predictG function that we built before, but this function for the stages after groups, like Round16, Quarters, Semi, Final

In [52]:
# a list that saved in it the winners on each stage
winners=[]
def predict(stage):
    """
    returns the result of each match in the knockout stages
    
    input:
        stage: Dataframe of the teams and thier knockout stage
    """
        
    for group in stage['Group'].drop_duplicates():
        print('_Starting Match {}:_'.format(group))

        for home, away in combinations(stage.query('Group == "{}"'.format(group)).values, 2):
            print("{} vs. {}: ".format(home[0], away[0]), end='')

            home = home[0]
            away = away[0]

            # Create a row for each match
            row = pd.DataFrame(np.array([[np.nan, np.nan, np.nan, True, True]]), columns=X_train.columns)

            home_rank = rank_df.loc[home, 'rank']
            home_points = rank_df.loc[home, 'points']
            opp_rank = rank_df.loc[away, 'rank']
            opp_points = rank_df.loc[away, 'points']

            row['average_rank'] = (home_rank + opp_rank) / 2
            row['rank_difference'] = home_rank - opp_rank
            row['point_difference'] = home_points - opp_points

            # Model Output
            home_win_prob = model.predict_proba(row)[:,1][0]



            # Saving Model Output
            world_cup.loc[home, 'total_prob'] += home_win_prob
            world_cup.loc[away, 'total_prob'] += 1-home_win_prob

            # Determining Win / Lose
            points = 0
            if home_win_prob < 0.5  :
                print("{} wins with {:.2f}".format(away, 1-home_win_prob))
                # if the winner in away place we add it in winners list
                winners.append(away)
            if home_win_prob > 0.5 :
                print("{} wins with {:.2f}".format(home, home_win_prob))
                # if the winner in home place we add it in winners list
                winners.append(home)
            if home_win_prob==0.5:
                import random
                rn = random.randint(0,2)
                if rn==0:
                    print("{} wins with {:.2f}".format(home, home_win_prob))
                    winners.append(home)
                else:
                    print("{} wins with {:.2f}".format(away, 1-home_win_prob))
                    winners.append(away)

#### Now we will pass round 16 teams to the function to predict the winners

In [53]:
predict(round_16_df)

_Starting Match R1:_
Netherlands vs. USA: Netherlands wins with 0.83
_Starting Match R2:_
England vs. Senegal: England wins with 0.75
_Starting Match R3:_
Argentina vs. Denmark: Argentina wins with 0.66
_Starting Match R4:_
France vs. Poland: Poland wins with 0.55
_Starting Match R5:_
Spain vs. Croatia: Spain wins with 0.65
_Starting Match R6:_
Belgium vs. Japan: Japan wins with 0.56
_Starting Match R7:_
Brazil vs. Uruguay: Brazil wins with 0.57
_Starting Match R8:_
Portugal vs. Switzerland: Switzerland wins with 0.73


#### We can see now the winners in winners list

In [54]:
winners

['Netherlands',
 'England',
 'Argentina',
 'Poland',
 'Spain',
 'Japan',
 'Brazil',
 'Switzerland']

##### based on the position of each team it will determine the matches of quarters stage

In [55]:
quarters=[[winners[0],'Q1'],[winners[2],'Q1'],[winners[1],'Q2'],[winners[3],'Q2'],
         [winners[4],'Q3'],[winners[6],'Q3'],[winners[5],'Q4'],[winners[7],'Q4']]
quarters_df = pd.DataFrame(quarters, columns=['Team', 'Group'])
quarters_df
winners=[]

##### It will call function predict which takes DataFrame of teams that qualified to the quarters round

###### Disply the winning team 

In [56]:
predict(quarters_df)

_Starting Match Q1:_
Netherlands vs. Argentina: Argentina wins with 0.66
_Starting Match Q2:_
England vs. Poland: England wins with 0.82
_Starting Match Q3:_
Spain vs. Brazil: Spain wins with 0.54
_Starting Match Q4:_
Japan vs. Switzerland: Japan wins with 0.51


In [57]:
winners

['Argentina', 'England', 'Spain', 'Japan']

##### based on the position of each team it will determine the matches of semi-final stage

In [58]:
semi=[[winners[2],'S1'],[winners[0],'S1'],[winners[3],'S2'],[winners[1],'S2']]
semi_df = pd.DataFrame(semi, columns=['Team', 'Group'])
semi_df
winners=[]

   ### Question 2: Which team will be the black horse in the World Cup ?
#### Japan has made it to the semis so they are the black horse in the World Cup because they haven't reach a quarter final in thier history!

In [59]:
predict(semi_df)

_Starting Match S1:_
Spain vs. Argentina: Argentina wins with 0.65
_Starting Match S2:_
Japan vs. England: England wins with 0.96


##### Now in the last stage it will determine the winning team in FIFA-WORLD-CUP 2022

In [60]:
final=[[winners[1],'F1'],[winners[0],'F1']]
final_df = pd.DataFrame(final, columns=['Team', 'Group'])
final_df

,Team,Group
0,England,F1
1,Argentina,F1



 ### Question 3: Who will win the World Cup according to KICKOFF?
 #### As we can see here Argentina will win the World Cup!

In [61]:
# POST /prediction_final
predict(final_df)

_Starting Match F1:_
England vs. Argentina: Argentina wins with 0.80


In [62]:
#import pydantic

In [63]:
#import pickle

In [64]:
#filename='Kickoff_model.pkl'
#pickle.dump(random_forest,open(filename,'wb'))

In [65]:
#loaded_model=pickle.load(open('Kickoff_model.pkl','rb'))